#### Objective

This notebook is to execute below tasks:
1. Fetch the data from GCS bucket
2. Filter key columns, required for Embeddings
3. Clean Columns and generate String Concating columns
4. Save output file back to GCS
5. Create another file with Diagnostic columns - to be used for recommendations and save to GCS

#### Python Version

In [1]:
from platform import python_version

print(python_version())

#ToDO - Need VM with python 3.10

3.10.12


#### Install Libraries

In [7]:
# pip install ydata_profiling dtale

! pip install openai

distutils: /root/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /root/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 225 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 381 kB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 80 kB 9.8 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 4.2 MB/s  eta 0:00:01
     |████████████████████████████████| 74 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 58 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 67.8 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: anyio
    Found existing installation: anyio 3.3.4
    Uninstalling anyio-3.3.4:
      Successfully uninstalled anyio-3.3.4
  Attempting uninstall: dis

In [2]:
# ! pip install tiktoken

In [ ]:
# pip install google-cloud-aiplatform

#### Imports

In [2]:
# from utils.bq_utils import BQClient
import pandas as pd


#### Local python scripts path

In [3]:
import sys
sys.path.append("./keys")
sys.path.append("./utils")

#### Create BQ Client Object

In [5]:
# bq_client = BQClient()

#### Read Data From BQ and Store to Local

In [6]:
# # The ID of your GCS bucket
# bucket_name = "emopti_shared"

# # The ID of your GCS object
# source_blob_name = "aiipem_deidentified_data_1000_20231208.csv"
# # source_blob_name = "aiipem_deidentified_data_20231208.csv"
# # source_blob_name = "d2i_LOF_091123.txt"

# # The path to which the file should be downloaded
# destination_file_name = f"./data/{source_blob_name}"

# bq_client.download_blob(bucket_name, source_blob_name, destination_file_name)

#### Read data from Local and Filter required columns

In [12]:
print("Original DF: ")
df = pd.read_csv("./data/deidentified/aiipem_deidentified_data_20231208.csv")
display(df.head(2))

#### all columns
all_columns = df.columns.to_list()
        
id_column = ['PT_Visit_ID_Hashed']

columns_for_embeddings = ['Pt_Arrival_Method', 'Pt_Complaint', 'Pt_Triage_Acuity',
           'Pt_Gender', 'Pt_Race', 'Pt_Ethnicity', 'Pulse', 'Resp', 'SpO2', 'Temp', 
           'Arrival_TimeOfDay', 'Age_Group',
           'grouped_arrival_method', 'bp_group']

columns_for_output = ['EHR_Disposition', 'd2i_Grouped_Disposition', 'emopti_grouped_disposition']
columns_to_keep = id_column + columns_for_embeddings + columns_for_output
columns_dropped = list(set(all_columns) - set(columns_to_keep))

print(f"\n\nID Columns :{id_column}")
print(f"\nColumns For Embeddings :{columns_for_embeddings}")
print(f"\nColumns for output :{columns_for_output}")
print(f"\nColumns dropped :{columns_dropped}")
print(f"\nColumns to keep :{columns_to_keep}")

print("\n\nFiltered DF: ")

#### Save everything
# df = df[columns_to_keep]
display(df.head(2))

Original DF: 


,PT_Visit_ID_Hashed,PT_ID_Hashed,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pt_Prime_ICD10,Pt_Secondary_ICD10,...,Temp,Visit_Duration_Hrs,Arrival_TimeOfDay,Age_Group,Pt_DOB_Year,Systolic_BP,Diastolic_BP,emopti_grouped_disposition,grouped_arrival_method,bp_group
0,c1cf913b612ffce254b33e996aa49b0def330b033058a7...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,O26.893,R10.2,...,97.9,1.5,12pm-6pm,20-40,1988.0,111.0,62.0,Left Against Medical Advice,ambulance,normal
1,094a742f233f2699fe3125b34ee61421e3f5ae54e26ca2...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,E86.0,D62,...,97.9,4.5,6pm-Midnight,20-40,1988.0,115.0,56.0,Discharged,private transport,normal




ID Columns :['PT_Visit_ID_Hashed']

Columns For Embeddings :['Pt_Arrival_Method', 'Pt_Complaint', 'Pt_Triage_Acuity', 'Pt_Gender', 'Pt_Race', 'Pt_Ethnicity', 'Pulse', 'Resp', 'SpO2', 'Temp', 'Arrival_TimeOfDay', 'Age_Group', 'grouped_arrival_method', 'bp_group']

Columns for output :['EHR_Disposition', 'd2i_Grouped_Disposition', 'emopti_grouped_disposition']

Columns dropped :['Pt_Prime_ICD10', 'Pt_Third_ICD10', 'PT_ID_Hashed', 'Diastolic_BP', 'Visit_Duration_Hrs', 'Pt_Secondary_ICD10', 'Systolic_BP', 'BP', 'Pt_DOB_Year']

Columns to keep :['PT_Visit_ID_Hashed', 'Pt_Arrival_Method', 'Pt_Complaint', 'Pt_Triage_Acuity', 'Pt_Gender', 'Pt_Race', 'Pt_Ethnicity', 'Pulse', 'Resp', 'SpO2', 'Temp', 'Arrival_TimeOfDay', 'Age_Group', 'grouped_arrival_method', 'bp_group', 'EHR_Disposition', 'd2i_Grouped_Disposition', 'emopti_grouped_disposition']


Filtered DF: 


,PT_Visit_ID_Hashed,PT_ID_Hashed,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pt_Prime_ICD10,Pt_Secondary_ICD10,...,Temp,Visit_Duration_Hrs,Arrival_TimeOfDay,Age_Group,Pt_DOB_Year,Systolic_BP,Diastolic_BP,emopti_grouped_disposition,grouped_arrival_method,bp_group
0,c1cf913b612ffce254b33e996aa49b0def330b033058a7...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,O26.893,R10.2,...,97.9,1.5,12pm-6pm,20-40,1988.0,111.0,62.0,Left Against Medical Advice,ambulance,normal
1,094a742f233f2699fe3125b34ee61421e3f5ae54e26ca2...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,E86.0,D62,...,97.9,4.5,6pm-Midnight,20-40,1988.0,115.0,56.0,Discharged,private transport,normal


#### Clean and Prepare data for Embeddings

In [13]:
print("DF for Embeddings: ")
# df_for_embeddings = df[columns_for_embeddings].copy()
df_for_embeddings = df.copy()

display(df_for_embeddings.head(2))

DF for Embeddings: 


,PT_Visit_ID_Hashed,PT_ID_Hashed,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pt_Prime_ICD10,Pt_Secondary_ICD10,...,Temp,Visit_Duration_Hrs,Arrival_TimeOfDay,Age_Group,Pt_DOB_Year,Systolic_BP,Diastolic_BP,emopti_grouped_disposition,grouped_arrival_method,bp_group
0,c1cf913b612ffce254b33e996aa49b0def330b033058a7...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,O26.893,R10.2,...,97.9,1.5,12pm-6pm,20-40,1988.0,111.0,62.0,Left Against Medical Advice,ambulance,normal
1,094a742f233f2699fe3125b34ee61421e3f5ae54e26ca2...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,E86.0,D62,...,97.9,4.5,6pm-Midnight,20-40,1988.0,115.0,56.0,Discharged,private transport,normal


#### EDA

In [14]:
df_for_embeddings['grouped_arrival_method'].value_counts()[0:20]

grouped_arrival_method
private transport    419143
ambulance             95477
helicopter            14108
police                 3831
other                  2882
Name: count, dtype: int64

In [29]:
# d = dtale.show(df_for_embeddings)
# print(d._main_url)


In [30]:
# ProfileReport(df_for_embeddings)

In [26]:
for col in df_for_embeddings.columns:
    print(f"\n\nColumn: {col}")
    print(f"\n{df_for_embeddings[col].value_counts()}")



Column: PT_Visit_ID_Hashed

PT_Visit_ID_Hashed
c1cf913b612ffce254b33e996aa49b0def330b033058a782cb0d7e5de368a121    1
69e4105bf4f0d1f749b69786ce57ed14cfcc988219445f26c67847ddb6bc9336    1
04c34b18b64d9ca80e53fb995e267f6112b342b2e4974e95a521ca1e44888be7    1
9a5d7f2f9d9700f0ac250343689ac95ba742ad9451076545ac76806ae8dac882    1
ea0ff264f45c7d01f965f4a74939db774ffdf2ca47a412a1143da031d5666182    1
                                                                   ..
8e11d91598d19a1b36107d043894d1c265bf983810fd0300cd4e9108bdedd98b    1
4a5b5d4661d2d0c0f79dc21a012ccfee2886a8ea787210b91a5312df336ebd27    1
4378143d798b52514a8588553bfc1c503caff5e2e16065a60ef998b1b1d07a36    1
c73a1b9d2b13cc02a1699279b1c31921d21379a01e80d27c4b59e1aca129e831    1
a84df23fbb604373367b4174e8c303717ce6868961fa17b94fadde1ceb4e76f7    1
Name: count, Length: 531127, dtype: int64


Column: PT_ID_Hashed

PT_ID_Hashed
21a5eb45e5f94f4a2bec7f6179b1cee5187164281e26277b918131fe314fd282    172
22acd581d47e0b4f2be0255574d1f5

#### Include 'Diastolic_BP' and 'Systolic_BP', "Arrival Method' (could be tried)

#### Concate Columns

In [27]:
df_for_embeddings.columns

Index(['PT_Visit_ID_Hashed', 'PT_ID_Hashed', 'Pt_Arrival_Method',
       'Pt_Complaint', 'Pt_Triage_Acuity', 'Pt_Gender', 'Pt_Race',
       'Pt_Ethnicity', 'Pt_Prime_ICD10', 'Pt_Secondary_ICD10',
       'Pt_Third_ICD10', 'EHR_Disposition', 'd2i_Grouped_Disposition', 'BP',
       'Pulse', 'Resp', 'SpO2', 'Temp', 'Visit_Duration_Hrs',
       'Arrival_TimeOfDay', 'Age_Group', 'Pt_DOB_Year', 'Systolic_BP',
       'Diastolic_BP', 'emopti_grouped_disposition', 'grouped_arrival_method',
       'bp_group', 'combined'],
      dtype='object')

In [15]:
df_for_embeddings["combined"] = ("Complaint: " + df_for_embeddings['Pt_Complaint'].str.strip() + 
                #   "; Arrival_Method: " + df_for_embeddings['Pt_Arrival_Method'].str.strip() + 
                  "; Triage_Acuity: " + df_for_embeddings['Pt_Triage_Acuity'].astype(str).str.strip() + 
                  "; Gender: " + df_for_embeddings['Pt_Gender'].str.strip() + 
                  "; Race: " + df_for_embeddings['Pt_Race'].str.strip() + 
                  "; Ethnicity: " + df_for_embeddings['Pt_Ethnicity'].str.strip() + 
                  "; Pulse: " + df_for_embeddings['Pulse'].astype(str).str.strip() + 
                  "; Resp: " + df_for_embeddings['Resp'].astype(str).str.strip() + 
                  "; SpO2: " + df_for_embeddings['SpO2'].astype(str).str.strip() + 
                  "; Temp: " + df_for_embeddings['Temp'].astype(str).str.strip() + 
                  "; Arrival_TimeOfDay: " + df_for_embeddings['Arrival_TimeOfDay'].str.strip() + 
                  "; Age_Group: " + df_for_embeddings['Age_Group'].str.strip() + 
                  "; Grouped_Arrival_Method: " + df_for_embeddings['grouped_arrival_method'].str.strip() + 
                  "; BP_Group: " + df_for_embeddings['bp_group'].str.strip()
                  )
display(df_for_embeddings.head(2))

,PT_Visit_ID_Hashed,PT_ID_Hashed,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pt_Prime_ICD10,Pt_Secondary_ICD10,...,Visit_Duration_Hrs,Arrival_TimeOfDay,Age_Group,Pt_DOB_Year,Systolic_BP,Diastolic_BP,emopti_grouped_disposition,grouped_arrival_method,bp_group,combined
0,c1cf913b612ffce254b33e996aa49b0def330b033058a7...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,O26.893,R10.2,...,1.5,12pm-6pm,20-40,1988.0,111.0,62.0,Left Against Medical Advice,ambulance,normal,Complaint: FLANK PAIN; Triage_Acuity: 3.0; Gen...
1,094a742f233f2699fe3125b34ee61421e3f5ae54e26ca2...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,E86.0,D62,...,4.5,6pm-Midnight,20-40,1988.0,115.0,56.0,Discharged,private transport,normal,Complaint: DIZZINESS; Triage_Acuity: 3.0; Gend...


In [16]:
print(f"Sample Patient info: \n{df_for_embeddings['combined'].iloc[0]}")

Sample Patient info: 
Complaint: FLANK PAIN; Triage_Acuity: 3.0; Gender: F; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 90.0; Resp: 20.0; SpO2: 99.0; Temp: 97.9; Arrival_TimeOfDay: 12pm-6pm; Age_Group: 20-40; Grouped_Arrival_Method: ambulance; BP_Group: normal


In [17]:
print(f"DF shape before drop_NA: {df_for_embeddings.shape}")

df_for_embeddings.dropna(subset = ['combined'], inplace=True)
print(f"DF shape after drop_NA: {df_for_embeddings.shape}")


DF shape before drop_NA: (535441, 28)


DF shape after drop_NA: (531127, 28)


#### Embeddings Model

In [4]:
# imports
from keys import OPENAI_KEY
from embeddings_utils import get_embedding, cosine_similarity, get_embeddings

# from keys.keys import OPENAI_KEY
# from utils.embeddings_utils import get_embedding, cosine_similarity, get_embeddings

import openai
# Set up your API credentials
openai.api_key = OPENAI_KEY

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

#### Approximate Tokens and Cost calculation

In [37]:
# import tiktoken

# encoding = tiktoken.get_encoding(embedding_encoding)
# n_tokens_list = []

# # omit reviews that are too long to embed
# df_for_embeddings["n_tokens"] = df_for_embeddings.combined.apply(lambda x: len(encoding.encode(x)))

# print(f"df len: {len(df_for_embeddings)}")
# display(df_for_embeddings.head(2))

# print(f"# Tokens: {df_for_embeddings['n_tokens'].sum()}")



In [38]:
# #### Cost
# n_tokens = df_for_embeddings['n_tokens'].sum()
# per_1k_price = 0.0001 
# print(f"approximate cost: ${round(n_tokens * per_1k_price/1000, 2)}")

# #### Cost for 100k comments
# n_tokens = df_for_embeddings[0:100000]['n_tokens'].sum()
# per_1k_price = 0.0001 
# print(f"approximate cost: ${round(n_tokens * per_1k_price/1000, 2)}")

In [39]:
# df_for_embeddings.to_csv("./data/deidentified/aiipem_deidentified_before_embeddings.csv.gz", index= False, compression='gzip')
# df_for_embeddings.head(2)

#### Generate Embeddings

In [29]:
#### Time taken in sequential order for 10k patients --> 50 seconds
import time

# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage
start_time= time.time()
df_copy = df_for_embeddings[0:10000].copy()
embedding_list = []
batch_size = 2048

for idx in range(0, len(df_copy)-1, batch_size):
    start_idx = idx
    end_idx = min(10000, idx+batch_size)
    print(f"embeddings for: {start_idx, end_idx}")
    txt_list = df_copy['combined'].iloc[start_idx:end_idx]
    embedding_values = get_embeddings(txt_list, model=embedding_model)
    embedding_list.extend(embedding_values)

print(f"time taken for embeddings : {time.time()- start_time}")

df_copy["embedding"]  = embedding_list

print(f"\nsaving embedings to disk:")
print(f"embedings df shape: {df_copy.shape}")

start_time= time.time()
df_copy.to_csv("./data/deidentified/aiipem_deidentified_embeddings_batch_0_to_10k.csv.gz", index= False, compression='gzip')
print(f"time taken to save embeddings: {time.time()- start_time}")


embeddings for: (0, 2048)
embeddings for: (2048, 4096)
embeddings for: (4096, 6144)
embeddings for: (6144, 8192)
embeddings for: (8192, 10000)
time taken for embeddings : 132.93656754493713
saving embedings to disk:
embedings df shape: (10000, 29)


In [30]:
display(df_copy.head(2))
df_copy.shape

,PT_Visit_ID_Hashed,PT_ID_Hashed,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pt_Prime_ICD10,Pt_Secondary_ICD10,...,Arrival_TimeOfDay,Age_Group,Pt_DOB_Year,Systolic_BP,Diastolic_BP,emopti_grouped_disposition,grouped_arrival_method,bp_group,combined,embedding
0,c1cf913b612ffce254b33e996aa49b0def330b033058a7...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,O26.893,R10.2,...,12pm-6pm,20-40,1988.0,111.0,62.0,Left Against Medical Advice,ambulance,normal,Complaint: FLANK PAIN; Triage_Acuity: 3.0; Gen...,"[0.009560050442814827, 0.002757575362920761, 0..."
1,094a742f233f2699fe3125b34ee61421e3f5ae54e26ca2...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,E86.0,D62,...,6pm-Midnight,20-40,1988.0,115.0,56.0,Discharged,private transport,normal,Complaint: DIZZINESS; Triage_Acuity: 3.0; Gend...,"[0.011276663281023502, 0.003159793559461832, 0..."


(10000, 29)

#### Read Embeddings from Disk

In [7]:
import pandas as pd
import time
import numpy as np
from ast import literal_eval

datafile_path = "./data/deidentified/aiipem_deidentified_embeddings_batch_0_to_10k.csv.gz"

start_time= time.time()
df_copy = pd.read_csv(datafile_path)
print(f"time takento read csv: {time.time()- start_time}")

start_time= time.time()
df_copy["embedding"] = df_copy.embedding.apply(literal_eval).apply(np.array)
print(f"time taken for literal_eval: {time.time()- start_time}")

display(df_copy.head(2))
df_copy.shape

time takento read csv: 13.978440284729004
time taken for literal_eval: 73.75912046432495


,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pulse,Resp,SpO2,Temp,Arrival_TimeOfDay,Age_Group,grouped_arrival_method,bp_group,combined,n_tokens,embedding
0,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,90.0,20.0,99.0,97.9,12pm-6pm,20-40,ambulance,normal,Complaint: FLANK PAIN; Triage_Acuity: 3.0; Gen...,97,"[0.009560050442814827, 0.002757575362920761, 0..."
1,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,104.0,18.0,100.0,97.9,6pm-Midnight,20-40,private transport,normal,Complaint: DIZZINESS; Triage_Acuity: 3.0; Gend...,98,"[0.011276663281023502, 0.003159793559461832, 0..."


(10000, 17)

#### Cosine Similarity

In [24]:
from utils.embeddings_utils import get_embedding, cosine_similarity

# search through the patients for a specific patient
def search_patients(df, patient_info):
    product_embedding = get_embedding(patient_info, model=embedding_model)
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    results = df.sort_values("similarity", ascending=False)
    return results

bold_s = "\033[1m"
bold_e = "\033[0m"

In [71]:
# 'Pt_Prime_ICD10' 'Pt_Secondary_ICD10', 'Pt_Third_ICD10'
# 'EHR_Disposition', 'd2i_Grouped_Disposition', 'emopti_grouped_disposition'


#### Example 1

In [31]:
top_k = 5
patient_id = 10005
df_patient_search = df_copy.copy()

patient_info = df_for_embeddings.iloc[patient_id]
results = search_patients(df_patient_search, patient_info['combined'])

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info['combined']}")
print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {patient_info['Pt_Prime_ICD10']}")
print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {patient_info['Pt_Secondary_ICD10']}")
print(f"{bold_s}Pt_Third_ICD10{bold_e}: {patient_info['Pt_Third_ICD10']}")

print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: F; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 108.0; Resp: 28.0; SpO2: 100.0; Temp: 98.4; Arrival_TimeOfDay: 12pm-6pm; Age_Group: 2-13; Grouped_Arrival_Method: private transport; BP_Group: normal
Pt_Prime_ICD10: R11.2
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Top 5 Matched Patients: 

Matched Patient 1: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: F; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 139.0; Resp: 35.0; SpO2: 97.0; Temp: 99.3; Arrival_TimeOfDay: 12pm-6pm; Age_Group: 2-13; Grouped_Arrival_Method: private transport; BP_Group: normal
Similariry: 0.9992981698019001
Pt_Prime_ICD10: K29.70
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: F; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 99.0; Resp: 16.0; SpO2: 98.0; Temp: 99.6; Arrival_TimeOfDay: 12pm-6pm; Age_Group: 20-40; Grouped_Arrival_Method: private transport; B

#### Example 2

In [75]:
df_patient_search = df_copy.copy()

patient_info = "Complaint: EMESIS"
results = search_patients(df_patient_search, patient_info)

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info}")
print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: EMESIS

Top 5 Matched Patients: 

Matched Patient 1: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: F; Race: WHITE OR CAUCASIAN; Ethnicity: Other; Pulse: 81.0; Resp: 24.0; SpO2: 100.0; Temp: 98.0; Arrival_TimeOfDay: 6pm-Midnight; Age_Group: 60-80; Grouped_Arrival_Method: private transport; BP_Group: high
Similariry: 0.874676077908779
Pt_Prime_ICD10: K43.6
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: F; Race: WHITE OR CAUCASIAN; Ethnicity: American; Pulse: 75.0; Resp: 16.0; SpO2: 100.0; Temp: 98.3; Arrival_TimeOfDay: 6am-12pm; Age_Group: 20-40; Grouped_Arrival_Method: private transport; BP_Group: normal
Similariry: 0.8742737063931766
Pt_Prime_ICD10: R11.2
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 3: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 57.0; Resp: 16.0; SpO2: 99.0; Temp: 97.8; Arrival_TimeOfDay: 12pm-6pm; A

#### Example 3

In [76]:
df_patient_search = df_copy.copy()

patient_info = "Complaint: EMESIS; Pulse: 130.0"
results = search_patients(df_patient_search, patient_info)

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info}")
print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: EMESIS; Pulse: 130.0

Top 5 Matched Patients: 

Matched Patient 1: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: M; Race: SOMETHING ELSE; Ethnicity: Other; Pulse: 75.0; Resp: 18.0; SpO2: 100.0; Temp: 98.2; Arrival_TimeOfDay: 12pm-6pm; Age_Group: 13-20; Grouped_Arrival_Method: private transport; BP_Group: normal
Similariry: 0.9021883769831781
Pt_Prime_ICD10: R11.2
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: EMESIS; Triage_Acuity: 3.0; Gender: F; Race: WHITE OR CAUCASIAN; Ethnicity: American; Pulse: 75.0; Resp: 16.0; SpO2: 100.0; Temp: 98.3; Arrival_TimeOfDay: 6am-12pm; Age_Group: 20-40; Grouped_Arrival_Method: private transport; BP_Group: normal
Similariry: 0.9015968277451051
Pt_Prime_ICD10: R11.2
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 3: Complaint: EMESIS; Triage_Acuity: 4.0; Gender: M; Race: SOMETHING ELSE; Ethnicity: Choose not to disclose; Pulse: 120.0; Resp: 40.0; SpO2: 100.0; Temp: 97.7; Arrival_Time

#### Example 4

In [77]:
top_k = 5
patient_id = 10010
df_patient_search = df_copy.copy()

patient_info = df_for_embeddings.iloc[patient_id]
results = search_patients(df_patient_search, patient_info['combined'])

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info['combined']}")
print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {patient_info['Pt_Prime_ICD10']}")
print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {patient_info['Pt_Secondary_ICD10']}")
print(f"{bold_s}Pt_Third_ICD10{bold_e}: {patient_info['Pt_Third_ICD10']}")

print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: DENTAL PAIN; Triage_Acuity: 5.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 72.0; Resp: 18.0; SpO2: 100.0; Temp: 97.5; Arrival_TimeOfDay: Midnight-6am; Age_Group: 13-20; Grouped_Arrival_Method: private transport; BP_Group: high
Pt_Prime_ICD10: nan
Pt_Secondary_ICD10: R03.0
Pt_Third_ICD10: nan

Top 5 Matched Patients: 

Matched Patient 1: Complaint: DENTAL PAIN; Triage_Acuity: 5.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 74.0; Resp: 16.0; SpO2: 99.0; Temp: 97.7; Arrival_TimeOfDay: Midnight-6am; Age_Group: 20-40; Grouped_Arrival_Method: private transport; BP_Group: high
Similariry: 0.9987675484025071
Pt_Prime_ICD10: K02.9
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: DENTAL PAIN; Triage_Acuity: 5.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 52.0; Resp: 20.0; SpO2: 100.0; Temp: 98.0; Arrival_TimeOfDay: 6pm-Midnight; Age_Group: 13-20; Grouped_Arrival_Metho

#### Example 5

In [78]:
top_k = 5
patient_id = 10110
df_patient_search = df_copy.copy()

patient_info = df_for_embeddings.iloc[patient_id]
results = search_patients(df_patient_search, patient_info['combined'])

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info['combined']}")
print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {patient_info['Pt_Prime_ICD10']}")
print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {patient_info['Pt_Secondary_ICD10']}")
print(f"{bold_s}Pt_Third_ICD10{bold_e}: {patient_info['Pt_Third_ICD10']}")

print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: MEDICATION REACTION; Triage_Acuity: 3.0; Gender: M; Race: WHITE OR CAUCASIAN; Ethnicity: Choose not to disclose; Pulse: 93.0; Resp: 17.0; SpO2: 100.0; Temp: 97.7; Arrival_TimeOfDay: 6am-12pm; Age_Group: 60-80; Grouped_Arrival_Method: ambulance; BP_Group: normal
Pt_Prime_ICD10: Z13.9
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Top 5 Matched Patients: 

Matched Patient 1: Complaint: MEDICATION REACTION; Triage_Acuity: 3.0; Gender: M; Race: WHITE OR CAUCASIAN; Ethnicity: Choose not to disclose; Pulse: 109.0; Resp: 18.0; SpO2: 99.0; Temp: 99.2; Arrival_TimeOfDay: 6pm-Midnight; Age_Group: 20-40; Grouped_Arrival_Method: ambulance; BP_Group: at_risk
Similariry: 0.9955260238216732
Pt_Prime_ICD10: R06.02
Pt_Secondary_ICD10: T40.425A
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: MEDICATION REACTION; Triage_Acuity: 3.0; Gender: F; Race: WHITE OR CAUCASIAN; Ethnicity: Choose not to disclose; Pulse: 75.0; Resp: 16.0; SpO2: 96.0; Temp: 97.9; Arrival_TimeOfDay: 6am-1

#### Example 6

In [79]:
top_k = 5
patient_id = 20000
df_patient_search = df_copy.copy()

patient_info = df_for_embeddings.iloc[patient_id]
results = search_patients(df_patient_search, patient_info['combined'])

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info['combined']}")
print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {patient_info['Pt_Prime_ICD10']}")
print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {patient_info['Pt_Secondary_ICD10']}")
print(f"{bold_s}Pt_Third_ICD10{bold_e}: {patient_info['Pt_Third_ICD10']}")

print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: CHEST PAIN; Triage_Acuity: 2.0; Gender: M; Race: WHITE OR CAUCASIAN; Ethnicity: Choose not to disclose; Pulse: 76.0; Resp: 15.0; SpO2: 98.0; Temp: 99.3; Arrival_TimeOfDay: Midnight-6am; Age_Group: 60-80; Grouped_Arrival_Method: ambulance; BP_Group: normal
Pt_Prime_ICD10: R07.9
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Top 5 Matched Patients: 

Matched Patient 1: Complaint: CHEST PAIN; Triage_Acuity: 2.0; Gender: F; Race: WHITE OR CAUCASIAN; Ethnicity: Choose not to disclose; Pulse: 77.0; Resp: 14.0; SpO2: 100.0; Temp: 98.4; Arrival_TimeOfDay: Midnight-6am; Age_Group: 80-100; Grouped_Arrival_Method: ambulance; BP_Group: high
Similariry: 0.9963186277495532
Pt_Prime_ICD10: R07.9
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: CHEST PAIN; Triage_Acuity: 2.0; Gender: M; Race: WHITE OR CAUCASIAN; Ethnicity: Choose not to disclose; Pulse: 105.0; Resp: 19.0; SpO2: 95.0; Temp: 97.6; Arrival_TimeOfDay: 6pm-Midnight; Age_Group: 40-60; Grou

#### Example 7

In [80]:
top_k = 5
patient_id = 50000
df_patient_search = df_copy.copy()

patient_info = df_for_embeddings.iloc[patient_id]
results = search_patients(df_patient_search, patient_info['combined'])

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info['combined']}")
print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {patient_info['Pt_Prime_ICD10']}")
print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {patient_info['Pt_Secondary_ICD10']}")
print(f"{bold_s}Pt_Third_ICD10{bold_e}: {patient_info['Pt_Third_ICD10']}")

print(f"\nTop {top_k} Matched Patients: \n")

for idx in range(0, top_k):
    
    matched_patient = results.iloc[idx]
    print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
    print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
    print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
    print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
    print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
    print()



Search Patient: Complaint: BACK PAIN; Triage_Acuity: 4.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 106.0; Resp: 17.0; SpO2: 97.0; Temp: 98.6; Arrival_TimeOfDay: 6pm-Midnight; Age_Group: 40-60; Grouped_Arrival_Method: ambulance; BP_Group: normal
Pt_Prime_ICD10: M54.50
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Top 5 Matched Patients: 

Matched Patient 1: Complaint: BACK PAIN; Triage_Acuity: 4.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 77.0; Resp: 16.0; SpO2: 100.0; Temp: 98.1; Arrival_TimeOfDay: 6am-12pm; Age_Group: 60-80; Grouped_Arrival_Method: ambulance; BP_Group: normal
Similariry: 0.997372208429384
Pt_Prime_ICD10: N20.1
Pt_Secondary_ICD10: nan
Pt_Third_ICD10: nan

Matched Patient 2: Complaint: BACK PAIN; Triage_Acuity: 3.0; Gender: M; Race: UNKNOWN; Ethnicity: Choose not to disclose; Pulse: 83.0; Resp: 20.0; SpO2: 97.0; Temp: 97.9; Arrival_TimeOfDay: 12pm-6pm; Age_Group: 40-60; Grouped_Arrival_Method: ambulance; BP_Group: no

## GCS Vector Search

#### Save the embeddings in a JSON file

In [5]:
print("Data with Embeddings: ")
display(df_copy.head(2))

print("\nFilter df for vector search: ")
df_filter = df_copy[["PT_Visit_ID_Hashed", "embedding"]]
df_filter.columns = ["id", "embedding"]
display(df_filter.head(2))

Data with Embeddings: 


NameError: name 'df_copy' is not defined

In [36]:
# First, export the id and embedding columns from the DataFrame in JSONL format, and save it.

# save id and embedding as a json file
jsonl_string = df_filter.to_json(orient = 'records', lines = True)
with open('./data/deidentified/aiipem_deidentified_data_20231208.json', 'w') as f:
  f.write(jsonl_string)


In [38]:
# show the first few lines of the json file
! head -n 1 ./data/deidentified/aiipem_deidentified_data_20231208.json

{"id":"c1cf913b612ffce254b33e996aa49b0def330b033058a782cb0d7e5de368a121","embedding":[0.0095600504,0.0027575754,0.0064827814,-0.0455709435,-0.0068247002,0.0127672497,-0.0222931094,-0.0193115771,-0.0409755558,-0.0174925681,-0.0165625494,0.0169318207,0.0030943656,0.0215955935,-0.0008684738,0.0267107002,0.0278321933,-0.0047800252,0.0110850092,-0.0112354532,-0.0109687569,0.0457077101,-0.0223204624,0.018367881,-0.0188328903,0.0071461038,-0.0013121136,-0.0213767663,-0.0069067609,-0.0356689729,0.0105516156,-0.0051322021,-0.0142238243,-0.0099840304,-0.0220195744,0.0069204373,-0.0048757629,-0.0245908033,0.0111260395,-0.0002120965,0.0241257939,0.0031046232,-0.0009300192,0.0163437203,-0.0038363293,0.0028601512,0.0163026899,-0.0126441587,-0.0315659493,0.0285297092,0.0039594201,-0.0112354532,-0.0166582856,0.0034191885,0.0309915245,-0.0195851121,0.027462922,0.0200501215,-0.0254934691,0.0039628395,-0.0006868295,-0.018914951,-0.0118850991,-0.0056143072,-0.0368998833,-0.004448364,-0.0091292327,0.021855

#### Copy the file to a new GCS bucket

In [11]:
BUCKET_NAME = "emopti_vector_search"
BUCKET_URI =  f"gs://{BUCKET_NAME}"
# ! gsutil -m cp ./data/deidentified/aiipem_deidentified_data_20231208.json {BUCKET_URI}

#### Create an Index

The parameters for creating index
- contents_delta_uri: The URI of Cloud Storage directory where you stored the embedding JSON files
- dimensions: Dimension size of each embedding. In this case, it is 768 as we are using the embeddings from the Text Embeddings API.
- approximate_neighbors_count: how many similar items we want to retrieve in typical cases
- distance_measure_type: what metrics to measure distance/similarity between embeddings. In this case it's DOT_PRODUCT_DISTANCE

In [9]:
# init the aiplatform package
from google.cloud import aiplatform

PROJECT_ID = "healthlabs-genesis"
aiplatform.init(project=PROJECT_ID)

# LOCATION = "us-central-1"
# aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [12]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
  display_name = f"{BUCKET_NAME}-index",
  contents_delta_uri = BUCKET_URI,
  dimensions = 1536,
  approximate_neighbors_count = 20,
  distance_measure_type = "DOT_PRODUCT_DISTANCE",
)

PermissionDenied: 403 Permission denied on resource project healthlabs-genesis. [links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
, reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "consumer"
  value: "projects/healthlabs-genesis"
}
]

#### Create Index Endpoint and deploy the Index

In [ ]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
  display_name = f"{BUCKET_NAME}-index-endpoint",
  public_endpoint_enabled = True,
)

In [ ]:
DEPLOYED_INDEX_ID = f"{BUCKET_NAME}_deployed"

# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(
  index = my_index, deployed_index_id = DEPLOYED_INDEX_ID
)

#### Run Query

In [ ]:

top_k = 5
patient_id = 10005
df_patient_search = df_copy.copy()

patient_info = df_for_embeddings.iloc[patient_id]
test_embeddings = [get_embedding(patient_info['combined'], model=embedding_model)]

print(f"\n{bold_s}Search Patient{bold_e}: {patient_info['combined']}")
print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {patient_info['Pt_Prime_ICD10']}")
print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {patient_info['Pt_Secondary_ICD10']}")
print(f"{bold_s}Pt_Third_ICD10{bold_e}: {patient_info['Pt_Third_ICD10']}")

# print(f"\nTop {top_k} Matched Patients: \n")

# for idx in range(0, top_k):
    
#     matched_patient = results.iloc[idx]
#     print(f"{bold_s}Matched Patient {idx+1}{bold_e}: {matched_patient['combined']}")
#     print(f"{bold_s}Similariry{bold_e}: {matched_patient['similarity']}")
#     print(f"{bold_s}Pt_Prime_ICD10{bold_e}: {matched_patient['Pt_Prime_ICD10']}")
#     print(f"{bold_s}Pt_Secondary_ICD10{bold_e}: {matched_patient['Pt_Secondary_ICD10']}")
#     print(f"{bold_s}Pt_Third_ICD10{bold_e}: {matched_patient['Pt_Third_ICD10']}")
#     print()


In [ ]:
# Test query
response = my_index_endpoint.find_neighbors(
  deployed_index_id = DEPLOYED_INDEX_ID,
  queries = test_embeddings,
  num_neighbors = 20,
)

# show the result
import numpy as np
for idx, neighbor in enumerate(response[0]):
  id = np.int64(neighbor.id)
  similar = df.query("id == @id", engine = "python")
  print(f"{neighbor.distance:.4f} {similar.title.values[0]}")

#### Get an existing Index

To get an Index object that already exists, replace the following [your-index-id] with the index ID and run the cell. You can check the ID on the Vector Search Console > INDEXES tab - https://console.cloud.google.com/vertex-ai/matching-engine/indexes

In [ ]:
my_index_id = f"{BUCKET_NAME}-index" # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

#### Get an existing Index Endpoint

To get an Index Endpoint object that already exists, replace the following [your-index-endpoint-id] with the Index Endpoint ID and run the cell. You can check the ID on the Vector Search Console > INDEX ENDPOINTS tab - https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints

In [ ]:
my_index_endpoint_id = f"{BUCKET_NAME}_deployed" # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)

#### IMPORTANT: Cleaning Up
In case you are using your own Cloud project, not a temporary project on Qwiklab, please make sure to delete all the Indexes, Index Endpoints and Cloud Storage buckets after finishing this tutorial. Otherwise the remaining objects would incur unexpected costs.

In [50]:
# # wait for a confirmation
# input("Press Enter to delete Index Endpoint, Index and Cloud Storage bucket:")

# # delete Index Endpoint
# my_index_endpoint.undeploy_all()
# my_index_endpoint.delete(force = True)

# # delete Index
# my_index.delete()

# # delete Cloud Storage bucket
# ! gsutil rm -r {BUCKET_URI}

## ToDo

#### Mike 
1. to add code snippets to clean the vitals like if Pulse is More than 100 add text "Tachycardic"
2. 'Pt_Prime_ICD10', 'Pt_Secondary_ICD10',  'Pt_Third_ICD10' - text mapping

#### Narayan
1. Move the Code to Worshop and Ping Mike - <b>Done</v>
1.2 Include 'Diastolic_BP' and 'Systolic_BP', "Arrival Method' (could be tried)
2. Store the embeddings to GCS
3. Create Google Vector search pipeline
4. Create two pipelines
    4.1 To Create embedings and store in GCS
    4.2 Code to search the nearest patient - to return 2 information, patient ids of top_k patients, (what information can we get from ANN search (how far the matched patients are from the current patient), cosine similarity or something else which tells how close the match are, embdeings as well if possible

#### Omar/Florin
1. Create a frontend app steamlite app or any other 
2. What Diagnostics was taken for the patients 


#### Ideas:  Help Dr. Pickup and Not miss Rare info

5k with Heart attacks
and only 10 with 'diasection'


